In [14]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
from numpy import random
import time
from sklearn import metrics

def tansig(x):
    return (2/(1+np.exp(-2*x)))-1

def pinv(A,reg):
    return np.mat(reg*np.eye(A.shape[1])+A.T.dot(A)).I.dot(A.T)


'''
参数压缩
'''
def shrinkage(a,b):
    z = np.maximum(a - b, 0) - np.maximum( -a - b, 0)
    return z
'''
参数稀疏化
'''
def sparse_bls(A,b):
    lam = 0.001
    itrs = 50
    AA = np.dot(A.T,A)   
    m = A.shape[1]
    n = b.shape[1]
    wk = np.zeros([m,n],dtype = 'double')
    ok = np.zeros([m,n],dtype = 'double')
    uk = np.zeros([m,n],dtype = 'double')
    L1 = np.mat(AA + np.eye(m)).I
    L2 = np.dot(np.dot(L1,A.T),b)
    for i in range(itrs):
        tempc = ok - uk
        ck =  L2 + np.dot(L1,tempc)
        ok = shrinkage(ck + uk, lam)
        uk += ck - ok
        wk = ok
    return wk
def bls_regression(train_x,train_y,test_x,test_y,s,C,NumFea,NumWin,NumEnhan):
   
    u = 0
    WF = list()
    for i in range(NumWin):
        random.seed(i+u)
        WeightFea=2*random.randn(train_x.shape[1]+1,NumFea)-1;
        WF.append(WeightFea)
#    random.seed(100)
    WeightEnhan=2*random.randn(NumWin*NumFea+1,NumEnhan)-1;
    time_start = time.time()
    H1 = np.hstack([train_x, 0.1 * np.ones([train_x.shape[0],1])]);
    y = np.zeros([train_x.shape[0],NumWin*NumFea])
    WFSparse = list()
    distOfMaxAndMin = np.zeros(NumWin)
    meanOfEachWindow = np.zeros(NumWin)
    for i in range(NumWin):
        WeightFea = WF[i]
        A1 = H1.dot(WeightFea)        
        scaler1 = preprocessing.MinMaxScaler(feature_range=(-1, 1)).fit(A1)
        A1 = scaler1.transform(A1)
        WeightFeaSparse  = sparse_bls(A1,H1).T
        WFSparse.append(WeightFeaSparse)
    
        T1 = H1.dot(WeightFeaSparse)
        meanOfEachWindow[i] = T1.mean()
        distOfMaxAndMin[i] = T1.max() - T1.min()
        T1 = (T1 - meanOfEachWindow[i])/distOfMaxAndMin[i] 
        y[:,NumFea*i:NumFea*(i+1)] = T1

    H2 = np.hstack([y,0.1 * np.ones([y.shape[0],1])])
    T2 = H2.dot(WeightEnhan)
    T2 = tansig(T2);
    T3 = np.hstack([y,T2])
    WeightTop = pinv(T3,C).dot(train_y)

    Training_time = time.time()- time_start
    print('Training has been finished!');
    print('The Total Training Time is : ', round(Training_time,6), ' seconds' )
    NetoutTrain = T3.dot(WeightTop)

    RMSE = np.sqrt((NetoutTrain-train_y).T*(NetoutTrain-train_y)/train_y.shape[0]) #均方根误差
    MAPE = sum(abs(NetoutTrain-train_y))/train_y.mean()/train_y.shape[0]  #平均绝对百分比误差
    RR2=metrics.r2_score(NetoutTrain,train_y)#决定系数
    
    train_RMSE = RMSE
    train_MAPE = MAPE
    train_RR2=RR2
    
    print('Training RMSE is : ',RMSE);
    print('Training MAPE is : ', MAPE)
    print('Training RR2 is : ',RR2)
    
    time_start = time.time()
    HH1 = np.hstack([test_x, 0.1 * np.ones([test_x.shape[0],1])])
    yy1=np.zeros([test_x.shape[0],NumWin*NumFea])
    for i in range(NumWin):
        WeightFeaSparse = WFSparse[i]
        TT1 = HH1.dot(WeightFeaSparse)
        TT1  = (TT1 - meanOfEachWindow[i])/distOfMaxAndMin[i]   
        yy1[:,NumFea*i:NumFea*(i+1)] = TT1

    HH2 = np.hstack([yy1, 0.1 * np.ones([yy1.shape[0],1])])
    TT2 = tansig(HH2.dot( WeightEnhan));
    TT3 = np.hstack([yy1,TT2])
    NetoutTest = TT3.dot(WeightTop)
    
    RMSE = np.sqrt((NetoutTest-test_y).T*(NetoutTest-test_y)/test_y.shape[0]);
    MAPE = sum(abs(NetoutTest-test_y))/test_y.mean()/test_y.shape[0]
    RR2=metrics.r2_score(NetoutTest,test_y)
    
    test_RMSE = RMSE
    test_MAPE = MAPE
    test_RR2=RR2
    
#%% Calculate the testing accuracy
    Testing_time = time.time() - time_start
    print('Testing has been finished!');
    print('The Total Testing Time is : ', round(Testing_time,6), ' seconds' );
    print('Testing RMSE is : ', RMSE) 
    print('Testing MAPE is : ', MAPE)
    print('Testing RR2 is : ',RR2)
    return test_RR2, test_RMSE, test_MAPE,train_RR2,train_RMSE,train_MAPE

In [15]:
datatest=pd.read_csv(r"C:\Users\ZHUO\Desktop\test_fsMIR_scaler.csv",index_col=0)
# datatest.shape #1365,177
# datatest.head()
datatrain=pd.read_csv(r"C:\Users\ZHUO\Desktop\train_fsMIR_scaler.csv",index_col=0)
# datatrain.shape  #3182,177
# datatrain.head()

result=datatest.append(datatrain)
result.shape  #4547,177
result.head()

,rest_time,heart_rate_stop,heart_rate_start,time_elapsed,total_running_time,time_since_last_stop,time_from_start_to_stop,altitude_start,distance_start,speed_start,...,avg_temp2,avg_temp_mean,avg_lat,avg_long,avg_alt,avg_hr,avg_speed,total_dist,avg_temp_mode,heart_rate_change_value
0,0.006724,0.486034,0.345,0.166214,0.162108,0.219250,0.049210,0.539456,0.163111,0.001084,...,0.838710,0.812975,0.735712,0.003189,0.659856,0.406908,0.133332,0.171063,0.838710,0.474308
1,0.018492,0.368715,0.240,0.044199,0.029094,0.019443,0.000974,0.580570,0.045203,0.005660,...,0.677419,0.641900,0.735557,0.003239,0.696870,0.633106,0.339680,0.038229,0.677419,0.474308
2,0.005764,0.625698,0.560,0.236305,0.159950,0.013376,0.002645,0.535809,0.295905,0.006744,...,0.693548,0.652492,0.735470,0.003278,0.674507,0.476990,0.356060,0.269436,0.693548,0.403162
3,0.002642,0.486034,0.470,0.096819,0.093826,0.124931,0.002158,0.566313,0.156488,0.000000,...,0.806452,0.794795,0.735548,0.003270,0.678807,0.514625,0.304041,0.124154,0.822581,0.375494
4,0.006244,0.765363,0.340,0.010162,0.010162,0.019443,0.006264,0.528515,0.004126,0.000000,...,0.596774,0.543685,0.735544,0.003270,0.679090,0.833955,0.454427,0.123098,0.596774,0.675889


In [16]:
data=np.array(result.iloc[:,:-1])
# data.shape 4547,176
labels=np.array(result.iloc[:,-1]).reshape(-1,1)
# labels

In [20]:
# 十折交叉验证BLS  
# 1 20 466 2 33 117
from sklearn.model_selection import KFold

NumFea = 18
NumWin = 14
NumEnhan = 23
s = 0.8
C = 2**-30


RRL = []
train_RRL = []
RMSEL = []
train_RMSEL = []
MAPEL = []
train_MAPEL = []
KF = KFold(n_splits = 10, shuffle = True, random_state = 520)
for k, (train, test) in enumerate(KF.split(data, labels)):
    train_x1 = data[train]
    test_x1 = data[test]
    train_y1 = labels[train]
    test_y1 = labels[test]
    test_RR2, test_RMSE, test_MAPE,train_RR2,train_RMSE,train_MAPE = bls_regression(train_x1,train_y1,test_x1,test_y1,s,C,NumFea,NumWin,NumEnhan)
    RRL.append(test_RR2)
    train_RRL.append(train_RR2)
    RMSEL.append(test_RMSE)
    train_RMSEL.append(train_RMSE)
    MAPEL.append(test_MAPE)
    train_MAPEL.append(train_MAPE)


print('test_RMSE_mean:',np.mean(RMSEL))
print('train_RMSE_mean:',np.mean(train_RMSEL))
print('test_RMSE_variance:',np.std(RMSEL))
print('train_RMSE_variance:',np.std(train_RMSEL))
print()

print('test_R2_mean:',np.mean(RRL))
print('train_R2_mean:',np.mean(train_RRL))
print('test_R2_variance:',np.std(RRL))
print('train_R2_variance:',np.std(train_RRL))
print()

print('test_MAPE_mean:',np.mean(MAPEL))
print('train_MAPE_mean:',np.mean(train_MAPEL))
print('test_MAPE_variance:',np.std(MAPEL))
print('train_MAPE_variance:',np.std(train_MAPEL))

Training has been finished!
The Total Training Time is :  0.31404  seconds
Training RMSE is :  [[0.00025942]]
Training MAPE is :  [[0.00041379]]
Training RR2 is :  0.9999949653798668
Testing has been finished!
The Total Testing Time is :  0.011291  seconds
Testing RMSE is :  [[0.00028236]]
Testing MAPE is :  [[0.00042834]]
Testing RR2 is :  0.9999937884156077
Training has been finished!
The Total Training Time is :  0.232469  seconds
Training RMSE is :  [[0.00064252]]
Training MAPE is :  [[0.00120177]]
Training RR2 is :  0.9999687434177604
Testing has been finished!
The Total Testing Time is :  0.001999  seconds
Testing RMSE is :  [[0.0006448]]
Testing MAPE is :  [[0.00118507]]
Testing RR2 is :  0.9999706470975964
Training has been finished!
The Total Training Time is :  0.233902  seconds
Training RMSE is :  [[0.00057055]]
Training MAPE is :  [[0.00101924]]
Training RR2 is :  0.9999757209923338
Testing has been finished!
The Total Testing Time is :  0.004109  seconds
Testing RMSE is : 